In [2]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch

device = torch.device('cpu')

model_pth = "checkpoints/deit-small-mengzi-l6-finetune/checkpoint-11000"
# model_pth = "models/tang-syn-5.0-online-epoch-1/"

model = VisionEncoderDecoderModel.from_pretrained(model_pth)
tokenizer = AutoTokenizer.from_pretrained(model_pth)
processor = TrOCRProcessor.from_pretrained(
            "microsoft/trocr-base-handwritten", size=224, resample=3,
            image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

model.to(device)

truth = """
test-niandai/1.png	华俊龙苦笑着说：文晋，这不公平：你的
test-niandai/2.png	都教一个班，我却教两个班，还是二年级，未
test-niandai/3.png	免有失公道吧！
test-niandai/4.png	李文晋笑着说：“是不公平，这分工作总得
test-niandai/5.png	有人去干吧？你说，该给谁干呢？我没说一句，
test-niandai/6.png	他们就告诉我：这是教务科的安排。廖组长还
test-niandai/7.png	说：“你们一起来的华俊龙老师，领导者得他起，
test-niandai/8.png	就把重担交给他了。”
test-niandai/9.png	张厚生笑着说：“这叫能者多劳，你就担起
test-niandai/10.png	这个危险吧，有什么困难，我们都会帮忙的。”
test-niandai/11.png	周宏义也说：“我们刚来，学校安排工作，
test-niandai/12.png	我们还是愉快地接受下来，不说什么为好。当
test-niandai/13.png	然俊龙一个人任二年级的课，而且教两个班，
test-niandai/14.png	比我们辛苦许多，从另一方面说，也是学校
test-niandai/15.png	对你的重视，不过，无论怎么说，还是辛苦老
""".strip().split("\n")

test_dir = "dataset/data/test-niandai"
# test_dir = "samples/images"
for file in os.listdir(test_dir):
    if not file.endswith((".jpg", ".png")):
        continue

    # print(truth.pop(0))

    image = Image.open(os.path.join(test_dir, file)).convert("RGB")

    pixel_values = (
        processor(image, return_tensors="pt").pixel_values).to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = tokenizer.decode(
        generated_ids[0], skip_special_tokens=True)
    print(generated_text.replace(" ", ""))

# text = "李文晋就走来了。他告诉大家说：“刚才我从廖"
# image = Image.open(os.path.join("sample_imgs", "0.png")).convert("RGB")
# # print(processor.image_processor.to_dict())
# pixel_values = processor(image, return_tensors="pt").pixel_values
# labels = tokenizer(text, return_tensors="pt").input_ids
# # outputs = model(pixel_values, labels=labels)
# outputs = model.generate(pixel_values)
# generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# print(labels, outputs, generated_text)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


华倾在苦笑着说：“文善，这不公平，你们
南部一个班，外部部西行班，还是二年级，来
免有关公道吧！”
李文秀笑着说：“是不公平，这分工作还得
事人去干吧，你说，该给给干呢。我说话一句，
他们就告诉我：这是教务科的安排。廖继昌还
说：你们一起来的今俊龙老师，领导着得他走，
就把香料交给他了。”
张厚生笑着说：“这哪能有多劳，你就把赵
这个危险吧，有什么困难，我们都会帮忙好。”
周宏之也说：我们刚幸，学校要用工作
则的还是愉快地接受下来，不论什么为好，各
丝理在一个人任二年级好课，而且敢两个班，
如此的会辜茫许多，从另一方面说，只是学校
对你的看视，不过，无论怎么说，还是辛苦苦


In [4]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch

device = torch.device('cpu')

model_pth = "models/tang-syn-2.0-checkpoint-374000"

tokenizer = AutoTokenizer.from_pretrained(model_pth)

res = tokenizer("你是我的小苹果 你是我的大苹\"““”‘’", max_length=64,
               truncation=True,)
print(res)
# tokenizer.decode(input_ids)
# tokenizer.decode([2, 17, 7158, 8884, 8884, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

{'input_ids': [101, 872, 3221, 2769, 4638, 2207, 5741, 3362, 872, 3221, 2769, 4638, 1920, 5741, 107, 100, 100, 100, 100, 100, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [1]:
import evaluate
import numpy as np

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    labels_str = tokenizer.batch_decode(labels_ids,
                                        skip_special_tokens=True)
    
    lines = map(lambda x: "\n".join(x).replace(" ", ""), zip(labels_str, pred_str))
    print("\n\n".join(lines))

    cer = cer_metric.compute(predictions=pred_str, references=labels_str)
    wer = wer_metric.compute(predictions=pred_str, references=labels_str)

    return {"cer": cer, "wer": wer}

In [2]:
import os
from os import path

import torch
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image

from data_aug_v2 import build_data_aug
# from torch.utils.data import Subset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from transformers import default_data_collator
from torch.utils.data import Subset

dataset_dir = 'dataset/data'
max_length = 64


def load_model(model_pth):
    model = VisionEncoderDecoderModel.from_pretrained(model_pth)
    model.cuda()
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained(model_pth)
    processor = TrOCRProcessor.from_pretrained(
            "microsoft/trocr-base-handwritten", size=384, resample=2,
            image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])
    return model, tokenizer, processor


class OCRDataset(Dataset):

    def __init__(self,
                 dataset_dir,
                 labels_dir,
                 labels,
                 transform,
                 processor,
                 tokenizer,
                 mode="train",
                 max_target_length=32,
                 device=None):
        self.dataset_dir = dataset_dir
        self.labels_dir = labels_dir
        self.labels = labels
        self.transform = transform
        self.device = device
        self.processor = processor
        self.mode = mode
        self.max_target_length = max_target_length
        self.tokenizer = tokenizer
        self.df = self.build_df()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        # Thirty percent of the time, use existing dataset
        text = self.df['text'][idx]
        # get file name + text
        file_name = self.df["file_name"][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(path.join(self.dataset_dir,
                                     file_name)).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        pixel_values = self.processor(image, return_tensors="pt").pixel_values

        # Remove spaces from text, as only the tang-syn version 1 had spaces before text
        text = text.strip()
        labels = self.tokenizer(text,
                                padding="max_length",
                                stride=32,
                                truncation=True,
                                max_length=self.max_target_length).input_ids

        # important: make sure that PAD tokens are ignored by the loss function
        labels = [
            label if label != self.tokenizer.pad_token_id else -100

            for label in labels
        ]

        encoding = {
            "pixel_values": pixel_values.squeeze(),
            "labels": torch.tensor(labels)
        }


        return encoding

    def build_df(self):
        li = []
        for root, _dirs, files in os.walk(self.labels_dir):
            for file in files:  # Loop through the dataset tsvfiles
                if not file.endswith(".tsv"):
                    continue

                if self.labels and file not in self.labels:
                    continue

                print(f"Processing {file}")
                li.append(
                    pd.read_table(path.join(root, file),
                                  names=["file_name", "text"]))

        return pd.concat(li, axis=0, ignore_index=True)


def build_eval_dataset(tokenizer, processor):
    eval_dataset = OCRDataset(
        dataset_dir=dataset_dir,
        labels_dir="dataset/labels/test",
        labels=["hwdb_ic13_3k.tsv"],
        tokenizer=tokenizer,
        processor=processor,
        mode="eval",
        transform=build_data_aug(64, "eval"),
        max_target_length=max_length
    )

    # # Create a random subset of the dataset
    # num_samples = 480

    # subset_indices = torch.randperm(len(eval_dataset))[:num_samples]
    # eval_dataset = Subset(eval_dataset, subset_indices.tolist())

    print("Number of validation examples:", len(eval_dataset))
    return eval_dataset


def init_trainer(model, tokenizer, compute_metrics, train_dataset,
                 eval_dataset):
    training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=48,
        per_device_eval_batch_size=48,
        num_train_epochs=3,
        fp16=True,
        learning_rate=4e-5,
        output_dir="./checkpoints",
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=100,
        save_strategy="steps",
        save_total_limit=5,
        save_steps=10000,
        eval_steps=10000,
        resume_from_checkpoint="./checkpoints/",
        dataloader_num_workers=4)

    # instantiate trainer
    return Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=default_data_collator
    )


for pth in os.listdir("models"):
    model_pth = os.path.join("models", pth)
    if not os.path.isdir(model_pth) or not pth.startswith("checkpoint"):
        continue
    
    # if not pth == "checkpoint-10400":
    #     continue

    print(pth)

    model, tokenizer, processor = load_model(model_pth)
    eval_dataset = build_eval_dataset(tokenizer=tokenizer, processor=processor)
    trainer = init_trainer(model, tokenizer, compute_metrics,
                           eval_dataset, eval_dataset)
    eval_result = None
    with torch.no_grad():
        eval_result = trainer.evaluate()
    
    print(eval_result)
    
    del model, tokenizer, processor
    torch.cuda.empty_cache()


/home/dotin13/.pyenv/versions/3.11.4/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dotin13/.pyenv/versions/3.11.4/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:63: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please sub

checkpoint-30200


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Processing hwdb_ic13_3k.tsv
Number of validation examples: 3431


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


...

Processing hwdb_ic13_3k.tsv
Number of validation examples: 3431


全球知名腕表品牌西铁城于2008年1月22日
合合球知名脑素品牌西铁城于2008年1月22日

在北京嘉里中心举行了“西铁城”真情行动“客
在在北京和国中心举行了“西铁城”真情行动“客

服体系”新闻发布会。西铁城进入中国市场已经
服体系”新闻发布会。而铁城进入中国市场的

超过半个世纪，深受中国消费者的喜爱和认同。为
场过半个世纪，深受中国消费者的喜爱和认同。为

回馈消费者长期以来的支持和关爱，西铁城在
回回馈消费者长期以来的支持和关爱，两铁城在

2008年伊始，正式发布“真情行动”客服体系，并
2006年伊始，正式发布“真情行动”，暑假结束，并

开通400-650-1958客服热线。据悉，西铁城
开开通40-60-1958客服模式。据悉，西铁城

是手表行业中首家发布客服体系的品牌，不仅
是可靠的业中首家发布影响和的品牌，不仅

显示了西铁城对其产品的高度自信，更表明了
星和了两块城对其产品的高度自信，更表明了

西铁城为中国消费者提供满意服务的决心。西
西铁成为中国消费者提供满意服务的决心。西

铁城（中国）钟表有限公司近期对客户服务部
映城（中国）钟表有限公司近期对客户股市

门做了调整和改组，并更名为客户满意部，希
门做了调整和改组，并更名为客户满意部，希

望通过不懈的努力，全方位地关注消费者的
望望通过不懈的努力，全方位地关注消费者的

各种需求，并秉承“以客户为中心”的服务
多种要求；并重点“以产为中心”的服务

理念，推出“真情行动”客服体系。
理念，推出“真情行动”系的体系。

万科集团在深圳新开的楼盘，价格就比同期的楼盘下跌了不少。董事长王石
万科集团在深圳新开的楼盘，价格就比同期的楼盖下跌了不少。董事长王石

也在央视奉劝大家现在不要买房，尽管他并没有提到房价究竟出现了什
也也在央视奉劝大家现在不要买房，尽管他并没有提到房价究竟出现了什

么样的变化。“房价的涨跌不能用统计数据说话，因为地段不同差异会很
么么样的变化。”房价的涨跌不能用统计数据说话，因为地段不同差异会很

大。”北京联达四方房地产经纪公司总经理杨少锋是坚定的房价下跌论
大。“北京联达四方房地产经纪公司总经理杨少锋是坚定的房价下跌论

的支持者。在他看来，判断房价只能看同一区域、同一楼盘出现的变化，从
的支持者。在他看来，判断房价只能看同一区域，同一楼盘出现的变化，从



Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Processing hwdb_ic13_3k.tsv
Number of validation examples: 3431


全球知名腕表品牌西铁城于2008年1月22日
合合球知名雕表品牌西铁城于2008年1月22日

在北京嘉里中心举行了“西铁城”真情行动“客
在在北京舞团中心举行了“西铁城”真情行动“客

服体系”新闻发布会。西铁城进入中国市场已经
服体系”新闻发布会。而缺城进入中国市场的

超过半个世纪，深受中国消费者的喜爱和认同。为
杨过半个世纪，深受中国消费者的喜爱和认同。为

回馈消费者长期以来的支持和关爱，西铁城在
回回馈消费者长期以来的支持和关爱，两铁城在

2008年伊始，正式发布“真情行动”客服体系，并
2008年伊始，正式发布“真情行动”客服体系，并

开通400-650-1958客服热线。据悉，西铁城
开开通40-60-1958客服模式。据悉，西铁城

是手表行业中首家发布客服体系的品牌，不仅
是是可表行业中首家发布案的体系的品牌，不仅

显示了西铁城对其产品的高度自信，更表明了
星和了两块城对其产品的高度自信，更表明了

西铁城为中国消费者提供满意服务的决心。西
西铁成为中国消费者提供满意服务的决心。西

铁城（中国）钟表有限公司近期对客户服务部
映城（中国）钟表有限公司近期对客户股市

门做了调整和改组，并更名为客户满意部，希
门做了调整和改组，并更名为客户满意部，希

望通过不懈的努力，全方位地关注消费者的
望望通过不懈的努力，全方位地关注消费者的

各种需求，并秉承“以客户为中心”的服务
各各种要求；并重点“以急产为中心”的服务

理念，推出“真情行动”客服体系。
理念，推出“真情行动”客服体系。

万科集团在深圳新开的楼盘，价格就比同期的楼盘下跌了不少。董事长王石
万科集团在深圳新开的楼盘，价格就比同期的楼盛下跌了不少。董事长王石

也在央视奉劝大家现在不要买房，尽管他并没有提到房价究竟出现了什
也也在央视奉劝大家现在不要买房，尽管他并没有提到房价究竟出现了什

么样的变化。“房价的涨跌不能用统计数据说话，因为地段不同差异会很
么么样的变化。”房价的涨跌不能用统计数据说话，因为地段不同差异会很

大。”北京联达四方房地产经纪公司总经理杨少锋是坚定的房价下跌论
大大。“北京联达四方房地产经纪公司总经理杨少锋是坚定的房价下跌论

的支持者。在他看来，判断房价只能看同一区域、同一楼盘出现的变化，从
的支持者。在他看来，判断房价只能看同一区域、同一楼盘出现的变化

tang-syn-1.0-checkpoint-280000

{'eval_loss': 1.519798755645752, 'eval_cer': 0.40390193507370764, 'eval_wer': 0.6983321165833661, 'eval_runtime': 377.7759, 'eval_samples_per_second': 9.082, 'eval_steps_per_second': 0.191}


tang-syn-2.0-checkpoint-374000

{'eval_loss': 0.6802470684051514, 'eval_cer': 0.19505134274475078, 'eval_wer': 0.32816308193407084, 'eval_runtime': 369.2777, 'eval_samples_per_second': 9.291, 'eval_steps_per_second': 0.195}


tang-syn-3.0-epoch-1

{'eval_loss': 1.134255051612854, 'eval_cer': 0.1194216060379564, 'eval_wer': 0.18192635758054293, 'eval_runtime': 241.6356, 'eval_samples_per_second': 14.199, 'eval_steps_per_second': 0.298}


checkpoint-30500

{'eval_loss': 1.2217962741851807, 'eval_cer': 0.15558527826639168, 'eval_wer': 0.24539317845664527, 'eval_runtime': 326.8407, 'eval_samples_per_second': 10.497, 'eval_steps_per_second': 0.22}


checkpoint-33600

{'eval_loss': 1.1907786130905151, 'eval_cer': 0.14627320291557314, 'eval_wer': 0.23189243891860634, 'eval_runtime': 327.7571, 'eval_samples_per_second': 10.468, 'eval_steps_per_second': 0.22}


checkpoint-51000

{'eval_loss': 1.2108386754989624, 'eval_cer': 0.1494708920372483, 'eval_wer': 0.23515085463907207, 'eval_runtime': 327.8426, 'eval_samples_per_second': 10.465, 'eval_steps_per_second': 0.22}


checkpoint-53200

{'eval_loss': 1.1683052778244019, 'eval_cer': 0.13790526073249, 'eval_wer': 0.2147885366043527, 'eval_runtime': 327.9052, 'eval_samples_per_second': 10.463, 'eval_steps_per_second': 0.22}


checkpoint-55400

{'eval_loss': 1.1703541278839111, 'eval_cer': 0.13339252898432627, 'eval_wer': 0.20729306836000488, 'eval_runtime': 327.7568, 'eval_samples_per_second': 10.468, 'eval_steps_per_second': 0.22}


checkpoint-48900

{'eval_loss': 1.155460000038147, 'eval_cer': 0.13623616816810066, 'eval_wer': 0.21204168103112733, 'eval_runtime': 326.5331, 'eval_samples_per_second': 10.507, 'eval_steps_per_second': 0.22}


checkpoint-57500

{'eval_loss': 1.1646119356155396, 'eval_cer': 0.13455021608285891, 'eval_wer': 0.2093726715672646, 'eval_runtime': 325.7347, 'eval_samples_per_second': 10.533, 'eval_steps_per_second': 0.221}


checkpoint-57600

{'eval_loss': 1.1678860187530518, 'eval_cer': 0.13510096043070455, 'eval_wer': 0.21078502240855862, 'eval_runtime': 325.3876, 'eval_samples_per_second': 10.544, 'eval_steps_per_second': 0.221}


checkpoint-57700

{'eval_loss': 1.17428719997406, 'eval_cer': 0.13770294648226097, 'eval_wer': 0.21508880016903725, 'eval_runtime': 325.9386, 'eval_samples_per_second': 10.527, 'eval_steps_per_second': 0.221}

checkpoint-57800

{'eval_loss': 1.1709500551223755, 'eval_cer': 0.13650030066145522, 'eval_wer': 0.21267557077879473, 'eval_runtime': 325.8716, 'eval_samples_per_second': 10.529, 'eval_steps_per_second': 0.221}


checkpoint-57900

{'eval_loss': 1.1812629699707031, 'eval_cer': 0.13939451840778685, 'eval_wer': 0.21737969995885278, 'eval_runtime': 325.4088, 'eval_samples_per_second': 10.544, 'eval_steps_per_second': 0.221}


checkpoint-58000

{'eval_loss': 1.1745655536651611, 'eval_cer': 0.13741071478748573, 'eval_wer': 0.21382102067370248, 'eval_runtime': 325.7256, 'eval_samples_per_second': 10.533, 'eval_steps_per_second': 0.221}


checkpoint-58100

{'eval_loss': 1.1712112426757812, 'eval_cer': 0.13623616816810066, 'eval_wer': 0.21241979070517455, 'eval_runtime': 325.9611, 'eval_samples_per_second': 10.526, 'eval_steps_per_second': 0.221}

In [3]:
eval_result

{'eval_loss': 1.1712112426757812,
 'eval_cer': 0.13623616816810066,
 'eval_wer': 0.21241979070517455,
 'eval_runtime': 315.1533,
 'eval_samples_per_second': 10.887,
 'eval_steps_per_second': 0.228}

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('0.log')
p.strip_dirs().sort_stats("cumtime").print_stats()

In [49]:
# Cleanup fonts that does not support cjk

import os
import pygame
import pygame.freetype
from fontTools.ttLib import TTFont

pygame.freetype.init()


def get_font_names(font_path):
    font = TTFont(font_path)
    names = []
    for record in font['name'].names:
        if record.nameID == 4:
            if record.getEncoding() == "x_mac_simp_chinese_ttx":
                names.append(record.string.decode('gbk'))
            else:
                names.append(record.toStr())

    return names


for file in os.listdir("fonts"):
    if file.lower().endswith((".ttf", ".otf")):
        # font = pygame.freetype.Font(f"fonts/{file}", size=10)
        # metrics = font.get_metrics("你是我的小苹果，我是你的大苹果！")

        # invalid_metrics = not all(metrics)

        names = get_font_names(f"fonts/{file}")
        invalid_type = any("篆" in font_name or (
            "繁" in font_name and "简" not in font_name) for font_name in names)

        if invalid_type:
            # print(names)
            os.rename(f"fonts/{file}", f"fonts/archives/{file}")